## 实验一：对 Breast Cancer 数据集进行数据探索与数据预处理操作

从以下网址下载数据集
http://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Coimbra


### 一、了解数据


1.打开下载的数据集文件，查看数据的前 5 行


In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv("./python_data/dataR2.csv")
df.head()

2.显示数据的列名、索引情况


In [ ]:
print(df.index, "\n")
print(df.columns)

3.显示数据的形状、基本信息、类型


In [ ]:
df.shape

In [ ]:
df.info()

4.调用 describe()函数进一步查看数据特征整体情况


In [ ]:
df.describe()

6.对数据集的含义及情况做大致描述（采用文字）


数据集的维度为 116 \* 10 的矩阵，共有 10 个属性（列）和 116 组数据（行），有 10 个预测变量，都是定量的和一个二元因变量


7.患病数据和健康数据分别是多少


In [ ]:
print("患病数据有：", "\n", df[df.iloc[:, 9] == 2].iloc[:, 9].count())
print()
print("健康数据有：", "\n", df[df.iloc[:, 9] == 1].iloc[:, 9].count())

8.年龄在 25-50 之间的数据


In [ ]:
df[(df.iloc[:, 0] >= 25) & (df.iloc[:, 0] <= 50)]

9.显示特征是 Age、BMI、MCP.1 的第 4-10 数据


In [ ]:
df.loc[3:9, ["Age", "BMI", "MCP.1"]]

### 二、数据预处理


#### 1.缺失值处理

检查数据的缺失值情况，若有缺失值，统计各特征缺失值多少情况，并做出合理处理，如值替换等


In [ ]:
df.isnull().sum()

#### 2.重复值检测，如有重复值，做删除处理


In [ ]:
df.drop_duplicates(keep=False)

#### 3.利用箱型图对 Glucose 做异常值检测,说明有无异常值，如有异常值，异常值个数是多少，分别是？


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 5), dpi=200)
df.loc[:, "Glucose"].plot.box()
plt.show()

df_Glucose = df.loc[:, "Glucose"]
df_Glucose.head()

IQR = df_Glucose.quantile(0.75) - df_Glucose.quantile(0.25)
val_low = df_Glucose.quantile(0.25) - 1.5 * IQR
val_up = df_Glucose.quantile(0.75) + 1.5 * IQR
print("IQR:{}".format(IQR))
print("Q_LOW:{}".format(val_low))
print("Q_UP:{}".format(val_up))

print()

print(
    "异常值共有:{}".format(
        df_Glucose[df_Glucose <= val_low].count()
        + df_Glucose[df_Glucose >= val_up].count()
    )
)
print()
print(df_Glucose[df_Glucose <= val_low])
print()
print(df_Glucose[df_Glucose >= val_up])

#### 4.对检测的异常值进行过滤


In [ ]:
df_change = df_Glucose[(df_Glucose >= val_low) & (df_Glucose <= val_up)]
df_change.describe()

#### 5.对 MCP.1 进行最小-最大规范化


In [ ]:
# （1）自编最小-最大规范化函数。
df_MCP = df.loc[:, "MCP.1"]
df_MCP.describe()
min = df_MCP.describe().iloc[3]
max = df_MCP.describe().iloc[7]
# print("min = ",min," max = ",max)
n = 0
for i in df_MCP:
    df.loc[n, "MCP.1"] = (i - min) / (max - min)
    n += 1
print(df_MCP)

#### 6.相关性，所有特征之间的相关性情况


In [ ]:
# 欧式距离（Eulidean distance）
def euclidean(x, y):
    return np.sqrt(np.sum((x - y) ** 2))


# 曼哈顿距离(Manhattan distance)
def manhattan(x, y):
    return np.sum(np.abs(x - y))


# 切比雪夫距离(Chebyshev distance)
def chebyshev(x, y):
    return np.max(np.abs(x - y))


# 闵可夫斯基距离(Minkowski distance)
def minkowski(x, y, p):
    return np.sum(np.abs(x - y) ** p) ** (1 / p)


# 汉明距离(Hamming distance)
def hamming(x, y):
    return np.sum(x != y) / len(x)


# 余弦距离
def cos_similarity(x, y):
    return np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))


from scipy.spatial.distance import pdist

x = df.iloc[:, 0]
y = df.iloc[:, 1]
X = np.vstack(
    [x, y]
)  # 按垂直方向（行顺序）堆叠数组构成一个新的数组，堆叠的数组需要具有相同的维度

print("Age与BMI的距离为：")
d1 = pdist(X, "euclidean")
print("欧式距离：", d1)

d2 = pdist(X, "cityblock")
print("曼哈顿距离：", d2)

d3 = pdist(X, "chebyshev")
print("切比雪夫距离：", d3)

d4 = pdist(X, "minkowski", p=2)
print("闵可夫斯基距离：", d4)

d5 = pdist(X, "cosine")
print("余弦相似性：", 1 - d5)

In [ ]:
# 通过热力图可视化数据相关性
from string import ascii_letters
import seaborn as sns

sns.heatmap(df.corr())

#### 7.离散化。

等距离散化
使用 Pandas 中的 cut 方法，对 BMI 实现等距离散化分组，根据中国标准 BMI 值: 正常:18.5-23.9、超重:≥24、偏胖:24 ～ 27.9、肥胖:≥28 划分


In [ ]:
BMI_groups1 = pd.cut(
    df["BMI"], bins=[0, 18.5, 23.9, 27.9, 100], labels=["偏瘦", "正常", "偏胖", "肥胖"]
)
# print(BMI_groups1)
print(df["Classification"].groupby(BMI_groups1).count())

#### 8.等频离散化，每箱的数据个数约为总数的四分之一


In [ ]:
BMI_groups2 = pd.qcut(df["BMI"], 4)
BMI_groups2

### 三.数据可视化

1. 绘制年龄分布直方图，bins=4。
2. bar 的宽度为 0.8,直方为蓝色。
3. 观察绘制的直方图，简要说明年龄分布情况。


In [ ]:
plt.rcParams["font.sans-serif"] = ["HeiTi TC"]

df_Age = df.loc[:, "Age"]
m1 = np.array(df_Age)
fre_tuple1 = plt.hist(m1, bins=4, rwidth=0.8, edgecolor="steelblue", color="blue")
plt.xlabel("Age")
plt.ylabel("Count")
plt.title("Age直方图")
x_loc = fre_tuple1[0]
y_loc = fre_tuple1[1]
# print(x_loc,'\n',y_loc)
for x, y in zip(x_loc, y_loc):
    plt.text(y + 8, x + 0.1, "%.0f" % x, ha="center", va="bottom", fontsize=13)

2.绘制 BMI 分布直方图，bins=[18,24,27,40],直方为绿色


In [ ]:
df_BMI = df.loc[:, "BMI"]
m2 = np.array(df_BMI)
fre_tuple2 = plt.hist(m2, bins=[18, 24, 27, 40], edgecolor="steelblue", color="green")
plt.xlabel("BMI")
plt.ylabel("Count")
plt.title("BMI直方图")
x_loc = fre_tuple2[0]
y_loc = fre_tuple2[1]
# print(x_loc,'\n',y_loc)
for x, y in zip(x_loc, y_loc):
    plt.text(y + 2, x + 0.1, "%.0f" % x, ha="center", va="bottom", fontsize=13)

3.调用 sns.countplot()函数，探索患病情况与年龄的关系。


In [ ]:
df["age_range"] = pd.cut(
    df["Age"], bins=[0, 18, 40, 66, 200], labels=["儿童", "青年", "中年", "老年"]
)
print(df.groupby(["Classification", "age_range"]).count())
ax = sns.countplot(x=df["age_range"], hue=df["Classification"])
ax.set_xlabel("年龄段")
ax.set_ylabel("数量")

### 四、数据降维


In [ ]:
import numpy as np
from sklearn.decomposition import PCA

X = np.array(df.iloc[:, 0:9], type(float))

pca = PCA(n_components=0.9, svd_solver="full")
# 如果n_components设为大于0小于1的小数且svd_solver设为’full’，则自动根据样本特征方差来决定降维到的维度数，这里n_components表示主成分的方差和所占的最小比例阈值。

newX = pca.fit_transform(X)

# print(X,'\n')

print(newX, "\n")

print(pca.explained_variance_ratio_)

In [ ]:
import random

fig = plt.figure(figsize=(18, 12))
ax = fig.add_subplot(111, projection="3d")
color = [
    "yellow",
    "black",
    "aqua",
    "green",
    "teal",
    "orange",
    "navy",
    "pink",
    "purple",
    "red",
]
for x in range(0, 115):
    ax.scatter(newX[x, 0], newX[x, 1], newX[x, 2], color=color[random.randint(0, 9)])